In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("seaborn-whitegrid")

np.random.seed(0)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#a. Importar o banco de dados utilizando o pandas
bancoTempo = pd.read_csv("/kaggle/input/szeged-weather/weatherHistory.csv")
display(bancoTempo)

In [ ]:
#b. Identificar o número de valores faltantes em cada colunas
bancoTempo.isnull().sum()

Coluna "Precip Type" retornando 517 ocorrências vazias, podendo significar que nesses respectivos dias não houve nenhum tipo de precipitação

In [ ]:
#c. Gerar momentos estatísticos do banco de dados
bancoTempo.describe()

Coluna "Loud Cover" saindo do padrão do banco, uma vez que está retornando apenas valores '0'

In [ ]:
#d. Utilizando o método “.hist” do pandas realize plots de distribuição dos valores das colunas
bancoTempo["Pressure (millibars)"].hist()
print(bancoTempo.loc[bancoTempo["Pressure (millibars)"] == 0, ["Pressure (millibars)"]].value_counts())
print("\n")
colunasNumericas = bancoTempo.select_dtypes(include = ["int64","float64"]).keys()
print("Colunas numéricas")
print(colunasNumericas)
bancoTempo[colunasNumericas].hist(figsize = (18,12))

Coluna "Pressure (millibars)" retornando ocorrências iguais a '0'

In [ ]:
#Deve-se agora definir quais são as features categóricas e quais são as numéricas,
# para isso existe o método “.select_dtypes”, aqui será necessário identificar apenas os
# nomes das features

bancoTempo.info()

colunasCategoricas = bancoTempo.select_dtypes(include = ["object"]).keys()
print("\nColunas Categóricas")
print(colunasCategoricas)

"Formatted Date" está como object, mas não é categórica

In [ ]:
#Inspecionando os valores únicos de cada feature categórica
print(bancoTempo["Precip Type"].value_counts())
print("")
print(bancoTempo["Daily Summary"].value_counts())
print("")
print(bancoTempo["Summary"].value_counts())

Os 3 últimos valores de "Summary" ocorrem apenas 1 vez cada, sendo necessário retirá-los

In [ ]:
#A partir das informações obtidas durante a exploração do banco de dados será agora
# necessário realizar sua limpeza. Aqui serão usadas pipelines que, após configuradas,
# realizarão a substituição dos valores faltantes, encoding das variáveis categóricas e, como
# saída, gerarão um objeto do tipo np.array, necessário para o treino da rede neural.

# Retirando a indicação de fuso-horário
bancoTempo["Date"] = bancoTempo["Formatted Date"].str.split(' ',expand = True)[0] + ' ' + bancoTempo["Formatted Date"].str.split(' ',expand = True)[1]
bancoTempo["Date"] = pd.to_datetime(bancoTempo["Date"])
bancoTempo.drop("Formatted Date", axis = 1, inplace = True)

# Adicionando colunas de hora, dia, mês e ano
bancoTempo["Year"] = bancoTempo["Date"].dt.year
bancoTempo["Month"] = bancoTempo["Date"].dt.month
bancoTempo["Day"] = bancoTempo["Date"].dt.day
bancoTempo["Hour"] = bancoTempo["Date"].dt.hour
bancoTempo.drop("Date", axis = 1, inplace = True)
bancoTempo.head()

In [ ]:
# Agora será criada a pipeline para completar a limpeza dos dados, com ela será
# realizada a substituição dos valores faltantes nas colunas “Pressure (millibars)” e
# “Precip Type” e o encoding, utilizando ordinal encoder, das variáveis categóricas. Para
# isso será utilizada a biblioteca Column Transformer do sklearn, mas antes deverão ser
# retirados os dados que não servirão para o treinamento do modelo.


from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

#Retirando a data e o target das colunas categóricas
colunasCategoricas = colunasCategoricas.drop(["Formatted Date", "Daily Summary"])

#Retirando colunas e linhas desnecessárias
bancoTempo.drop("Loud Cover", axis = 1, inplace = True)
bancoTempo.drop(bancoTempo.loc[bancoTempo["Summary"].isin(["Breezy and Dry","Windy and Dry","Dangerously Windy and Partly Cloudy"])].index, inplace=True)

#Criando imputers e encoder
imputer = SimpleImputer(missing_values=0)
cat_transf = Pipeline(steps=[("imputer",SimpleImputer(strategy = "constant", fill_value = "none")),
                             ("encoder",OrdinalEncoder())]) 

#Criando pipeline para realizar as trasnformações nas colunas
transformer = ColumnTransformer(transformers=[("press", imputer, ["Pressure (millibars)"]),
                                              ("categorical", cat_transf, colunasCategoricas)], remainder = "passthrough")

In [ ]:
#O último passo da limpeza dos dados será a separação em treino e validação,
# aplicação da pipeline de limpeza e o encoding da coluna que será o target (“Daily summary”)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Realizando a separação do banco de dados
X = bancoTempo.drop("Daily Summary", axis = 1)
y = bancoTempo["Daily Summary"]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, stratify = y)

# Aplicando pipeline
X_train = transformer.fit_transform(X_train)
X_test = transformer.transform(X_test)

# Realizando o encoding do target
label = LabelEncoder()
y_train = label.fit_transform(y_train)
y_test = label.transform(y_test)

In [ ]:
#Primeiramente deve-se criar a arquitetura da rede neural, ela terá duas camadas
# escondidas, sendo cada uma com uma saída de dimensão 256, entre elas serão
# utilizados Batch normalization e Dropout. Como funções de ativação serão usados
# “relu” para as camadas escondidas e o “softmax” na camada de saída, esta função
# serve para que o modelo realize predições na forma de classificação não binária, ou
# seja, que seja capaz de prever mais de duas classes.

from tensorflow import keras
from tensorflow.keras import layers

#Identificando dimensões de entrada e saída
input_size = [X.shape[1]]
output_size = y.unique().shape[0]

#Criando arquietura do modelo
model = keras.Sequential([layers.BatchNormalization(input_shape = input_size),
                          layers.Dense(256,activation = "relu"),
                          layers.BatchNormalization(),
                          layers.Dropout(0.3),
                          layers.Dense(256,activation = "relu"),
                          layers.BatchNormalization(),
                          layers.Dropout(0.3),
                          layers.Dense(output_size,activation = "softmax")])

#Realizando o tratamento do target
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [ ]:
#Por fim, será realizado o treinamento do modelo e plotagem dos resultados das
# avaliações, será utilizado o early stopping para evitar o overfitting.

model.compile(optimizer = "adam",
             loss = "categorical_crossentropy",
             metrics = ["categorical_accuracy"])

#a. Configurar early stopping - Utilizar a biblioteca do keras para configurar early stopping, com uma
# tolerância de 10 épocas e uma variação mínima no resultado da função
# de perda de 0.001
early_stopping = keras.callbacks.EarlyStopping(patience = 10,
                                               min_delta = 0.001,
                                               restore_best_weights = True)

#b. Realizar o treino e armazenar os resultados da avaliação do modelo
history = model.fit(X_train, y_train,
                    validation_data = (X_test, y_test),
                    batch_size = 3000,
                    epochs = 300,
                    callbacks = [early_stopping])

#c. Para a plotagem deve-se transformar os resultados do treino em um dataframe
# do pandas e plotar os valores de perda e acurácia em função das épocas

history = pd.DataFrame(history.history)
fig, axs = plt.subplots(1,2, figsize = (18,6))

axs[0].set_title("Cross-Entropy")
sns.lineplot(data = history.loc[:,["loss", "val_loss"]], ax = axs[0])

axs[1].set_title("Accuracy")
sns.lineplot(data = history.loc[:,["categorical_accuracy", "val_categorical_accuracy"]], ax = axs[1])